In [31]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import linregress
from scipy.stats import sem, ttest_ind
from matplotlib import pyplot as plt

%matplotlib notebook

In [32]:
# Read in AFL / CIO data (2700 records)

salary = "Resources/ceo_salary_aflcio.csv"
salary_df = pd.read_csv(salary, encoding="utf-8")
salary_df.head()

,Symbol,Name,CEO,YR,SALARY
0,TSLA,"Tesla, Inc.",Elon Musk,2018,2284044884
1,AAXN,"Axon Enterprise, Inc.",Patrick Smith,2018,246026710
2,DISCA,"Discovery, Inc.",David Zaslav,2018,129499005
3,ORCL,Oracle Corp.,Mark Hurd,2018,108295023
4,OMF,"OneMain Holdings, Inc.",Jay Levine,2018,71532583


In [33]:
#Import Company data

consfin_csv = "Resources/company_data.csv"
com_df = pd.read_csv(consfin_csv)
com_df.head()

,Symbol,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
0,MMM,3M Company,Industrials,222.89,24.31,2.332862,7.92,259.77,175.490,138721055226,9.048000e+09,4.390271,11.34,http://www.sec.gov/cgi-bin/browse-edgar?action...
1,AOS,A.O. Smith Corp,Industrials,60.24,27.76,1.147959,1.70,68.39,48.925,10783419933,6.010000e+08,3.575483,6.35,http://www.sec.gov/cgi-bin/browse-edgar?action...
2,ABT,Abbott Laboratories,Health Care,56.27,22.51,1.908982,0.26,64.60,42.280,102121042306,5.744000e+09,3.740480,3.19,http://www.sec.gov/cgi-bin/browse-edgar?action...
3,ABBV,AbbVie Inc.,Health Care,108.48,19.41,2.499560,3.29,125.86,60.050,181386347059,1.031000e+10,6.291571,26.14,http://www.sec.gov/cgi-bin/browse-edgar?action...
4,ACN,Accenture plc,Information Technology,150.51,25.47,1.714470,5.44,162.60,114.820,98765855553,5.643228e+09,2.604117,10.62,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [34]:
#Making a copy of the com_df dataframe because it is being merged twice into 2 different 
# datasets for different questions. Avoids conflict

com_df2 = com_df
com_df2.head()

,Symbol,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
0,MMM,3M Company,Industrials,222.89,24.31,2.332862,7.92,259.77,175.490,138721055226,9.048000e+09,4.390271,11.34,http://www.sec.gov/cgi-bin/browse-edgar?action...
1,AOS,A.O. Smith Corp,Industrials,60.24,27.76,1.147959,1.70,68.39,48.925,10783419933,6.010000e+08,3.575483,6.35,http://www.sec.gov/cgi-bin/browse-edgar?action...
2,ABT,Abbott Laboratories,Health Care,56.27,22.51,1.908982,0.26,64.60,42.280,102121042306,5.744000e+09,3.740480,3.19,http://www.sec.gov/cgi-bin/browse-edgar?action...
3,ABBV,AbbVie Inc.,Health Care,108.48,19.41,2.499560,3.29,125.86,60.050,181386347059,1.031000e+10,6.291571,26.14,http://www.sec.gov/cgi-bin/browse-edgar?action...
4,ACN,Accenture plc,Information Technology,150.51,25.47,1.714470,5.44,162.60,114.820,98765855553,5.643228e+09,2.604117,10.62,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [35]:
#Read in Fortune_1000 file

fortune = "Resources/fortune_1000_part.csv"
fortune_df = pd.read_csv(fortune)
fortune_df.head()

,Company,Revenues ($M),Profits ($M),Assets ($M),Employees,Sector,Industry,City,State
0,Walmart,"$500,343","$9,862.00","$204,522","2,300,000",Retailing,General Merchandisers,Bentonville,AR
1,Exxon Mobil,"$244,363","$19,710.00","$348,691","71,200",Energy,Petroleum Refining,Irving,TX
2,Berkshire Hathaway,"$242,137","$44,940.00","$702,095","377,000",Financials,Insurance: Property and Casualty (Stock),Omaha,NE
3,Apple,"$229,234","$48,351.00","$375,319","123,000",Technology,"Computers, Office Equipment",Cupertino,CA
4,UnitedHealth Group,"$201,159","$10,558.00","$139,058","260,000",Health Care,Health Care: Insurance and Managed Care,Minnetonka,MN


In [36]:
# Formatting the columns (revenue,employees) to be able to work with them.

fortune_df["revenue"] = pd.to_numeric(fortune_df["Revenues ($M)"].replace({'\$':'',',':'', '&':''},regex = True))
fortune_df["employees"] = pd.to_numeric(fortune_df["Employees"].replace({'\$':'',',':''},regex = True))
fortune_df.to_csv("Resources/basic1000data.csv", index=False, header=True)

In [37]:
#Making a copy of the fortune_df dataframe because it is being merged twice into 2 different 
# datasets for different questions. Avoids conflicts

sector_data = fortune_df
sector_data.head()

,Company,Revenues ($M),Profits ($M),Assets ($M),Employees,Sector,Industry,City,State,revenue,employees
0,Walmart,"$500,343","$9,862.00","$204,522","2,300,000",Retailing,General Merchandisers,Bentonville,AR,500343.0,2300000
1,Exxon Mobil,"$244,363","$19,710.00","$348,691","71,200",Energy,Petroleum Refining,Irving,TX,244363.0,71200
2,Berkshire Hathaway,"$242,137","$44,940.00","$702,095","377,000",Financials,Insurance: Property and Casualty (Stock),Omaha,NE,242137.0,377000
3,Apple,"$229,234","$48,351.00","$375,319","123,000",Technology,"Computers, Office Equipment",Cupertino,CA,229234.0,123000
4,UnitedHealth Group,"$201,159","$10,558.00","$139,058","260,000",Health Care,Health Care: Insurance and Managed Care,Minnetonka,MN,201159.0,260000


In [38]:
fortune_df.rename(columns={'Company':'Name'}, inplace=True)
fortune_df.head()

,Name,Revenues ($M),Profits ($M),Assets ($M),Employees,Sector,Industry,City,State,revenue,employees
0,Walmart,"$500,343","$9,862.00","$204,522","2,300,000",Retailing,General Merchandisers,Bentonville,AR,500343.0,2300000
1,Exxon Mobil,"$244,363","$19,710.00","$348,691","71,200",Energy,Petroleum Refining,Irving,TX,244363.0,71200
2,Berkshire Hathaway,"$242,137","$44,940.00","$702,095","377,000",Financials,Insurance: Property and Casualty (Stock),Omaha,NE,242137.0,377000
3,Apple,"$229,234","$48,351.00","$375,319","123,000",Technology,"Computers, Office Equipment",Cupertino,CA,229234.0,123000
4,UnitedHealth Group,"$201,159","$10,558.00","$139,058","260,000",Health Care,Health Care: Insurance and Managed Care,Minnetonka,MN,201159.0,260000


In [39]:
#Merge two dataframes on Ticker symbol - Used for CEO Salary Question

salary_co = pd.merge(salary_df, com_df, on="Symbol")
salary_co.head()

,Symbol,Name_x,CEO,YR,SALARY,Name_y,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
0,DISCA,"Discovery, Inc.",David Zaslav,2018,129499005,Discovery Communications-A,Consumer Discretionary,22.87,11.00,0.000000,1.96,30.250,15.99,8763756733,2.100000e+09,1.804088,1.50,http://www.sec.gov/cgi-bin/browse-edgar?action...
1,ORCL,Oracle Corp.,Mark Hurd,2018,108295023,Oracle Corp.,Information Technology,46.84,18.81,1.555146,2.20,53.140,40.01,202302349740,1.654500e+10,5.256165,3.56,http://www.sec.gov/cgi-bin/browse-edgar?action...
2,DIS,The Walt Disney Co.,Robert Iger,2018,65645214,The Walt Disney Company,Consumer Discretionary,101.35,17.78,1.603665,5.70,116.100,96.20,157817273295,1.254400e+10,3.963556,3.82,http://www.sec.gov/cgi-bin/browse-edgar?action...
3,FOXA,"Twenty-First Century Fox, Inc.",James Murdoch,2018,50263861,Twenty-First Century Fox Class A,Consumer Discretionary,34.56,17.81,0.998336,1.59,39.135,24.81,44027094922,5.280000e+09,2.108032,4.11,http://www.sec.gov/cgi-bin/browse-edgar?action...
4,EL,"The Estee Lauder Companies, Inc.",Fabrizio Freda,2018,48753819,Estee Lauder Cos.,Consumer Staples,131.46,37.78,1.128350,3.35,138.740,81.69,49543264457,2.478000e+09,3.952258,10.82,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [40]:
# Used for CEO Salary Question - creating new dataframe with selected variables

sector_ceo_b = salary_co[["Symbol", "Name_x", "Sector", "Price/Earnings", "CEO", 'SALARY', 'Market Cap']]
sector_ceo_b.head() 

,Symbol,Name_x,Sector,Price/Earnings,CEO,SALARY,Market Cap
0,DISCA,"Discovery, Inc.",Consumer Discretionary,11.00,David Zaslav,129499005,8763756733
1,ORCL,Oracle Corp.,Information Technology,18.81,Mark Hurd,108295023,202302349740
2,DIS,The Walt Disney Co.,Consumer Discretionary,17.78,Robert Iger,65645214,157817273295
3,FOXA,"Twenty-First Century Fox, Inc.",Consumer Discretionary,17.81,James Murdoch,50263861,44027094922
4,EL,"The Estee Lauder Companies, Inc.",Consumer Staples,37.78,Fabrizio Freda,48753819,49543264457


In [41]:
#CEO Salary question

sector_ceo_b.rename(columns={'Name_x':'Company',
                          'SALARY':'Salary',
                          'Price/Earnings': 'PE_Ratio'}, 
                             inplace=True)
sector_ceo_b.head()

/anaconda3/envs/pythondatapython=3.6/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Symbol,Company,Sector,PE_Ratio,CEO,Salary,Market Cap
0,DISCA,"Discovery, Inc.",Consumer Discretionary,11.00,David Zaslav,129499005,8763756733
1,ORCL,Oracle Corp.,Information Technology,18.81,Mark Hurd,108295023,202302349740
2,DIS,The Walt Disney Co.,Consumer Discretionary,17.78,Robert Iger,65645214,157817273295
3,FOXA,"Twenty-First Century Fox, Inc.",Consumer Discretionary,17.81,James Murdoch,50263861,44027094922
4,EL,"The Estee Lauder Companies, Inc.",Consumer Staples,37.78,Fabrizio Freda,48753819,49543264457


In [42]:
sector_ceo_b.shape

(459, 7)

In [43]:
sector_ceo = sector_ceo_b.dropna(how = 'any')

In [44]:
sector_ceo.shape

(457, 7)

In [45]:
# Export dataframe for Question 4 on CEO Success as a CSV, without the Pandas index, but with the header
sector_ceo.to_csv("Resources/sector_ceo.csv", index=False, header=True)

In [46]:
#Merge Fortune_1000 data with company data to create dataset for Questions 2, 3

both_df = pd.merge(fortune_df, com_df2, on="Name")
both_df.head()

,Name,Revenues ($M),Profits ($M),Assets ($M),Employees,Sector_x,Industry,City,State,revenue,...,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
0,Berkshire Hathaway,"$242,137","$44,940.00","$702,095","377,000",Financials,Insurance: Property and Casualty (Stock),Omaha,NE,242137.0,...,30.43,0.000000,9.76,217.62,160.93,261401203633,0.000000e+00,1.432823,1.58,http://www.sec.gov/cgi-bin/browse-edgar?action...
1,CVS Health,"$184,765","$6,622.00","$95,131","203,000",Health Care,Health Care: Pharmacy and Other Services,Woonsocket,RI,184765.0,...,12.36,2.689980,4.91,84.00,66.45,75323141722,1.170400e+10,0.549852,2.13,http://www.sec.gov/cgi-bin/browse-edgar?action...
2,General Motors,"$157,311","($3,864.00)","$212,482","180,000",Motor Vehicles & Parts,Motor Vehicles and Parts,Detroit,MI,157311.0,...,6.58,3.585751,6.00,46.76,31.92,61536606173,2.354100e+10,0.563322,1.33,http://www.sec.gov/cgi-bin/browse-edgar?action...
3,Ford Motor,"$156,776","$7,602.00","$257,808","202,000",Motor Vehicles & Parts,Motor Vehicles and Parts,Dearborn,MI,156776.0,...,5.89,6.784387,1.90,13.48,10.19,42414328338,9.281000e+09,0.270541,1.26,http://www.sec.gov/cgi-bin/browse-edgar?action...
4,General Electric,"$122,274","($5,786.00)","$377,945","313,000",Industrials,Industrial Machinery,Boston,MA,122274.0,...,13.76,3.147541,-0.72,30.59,14.71,132249296250,-2.060000e+08,1.088761,1.70,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [47]:
#Dhruvs clean code that saves a dataframe as csv at the end. ----------------------------------

In [48]:
sector_data = pd.read_csv("Resources/fortune_1000_part.csv", thousands=',')
new_sector_data = sector_data.drop(columns=['Company', 'Revenues ($M)', 'Profits ($M)', 'Assets ($M)'], axis=1)
new_sector_data.head()

,Employees,Sector,Industry,City,State
0,2300000,Retailing,General Merchandisers,Bentonville,AR
1,71200,Energy,Petroleum Refining,Irving,TX
2,377000,Financials,Insurance: Property and Casualty (Stock),Omaha,NE
3,123000,Technology,"Computers, Office Equipment",Cupertino,CA
4,260000,Health Care,Health Care: Insurance and Managed Care,Minnetonka,MN


In [49]:
# Figure out the following: Biggest/most popular Sector by State
# Compare that by the # of employees per sector

#Bar graph for both above
#x axis: Sector Names

#Define biggest/most popular sector by total number of employees in each sector

In [50]:
new_sector_data["Employees"] = pd.to_numeric(sector_data["Employees"])

In [51]:
sorted_sector_data = new_sector_data.sort_values("Employees", ascending=False)
sorted_sector_data.head()

,Employees,Sector,Industry,City,State
0,2300000,Retailing,General Merchandisers,Bentonville,AR
7,566000,Retailing,Internet Services and Retailing,Seattle,WA
396,450000,"Hotels, Restaurants & Leisure",Food Services,Plano,TX
16,449000,Food & Drug Stores,Food and Drug Stores,Cincinnati,OH
22,413000,Retailing,Specialty Retailers: Other,Atlanta,GA


In [52]:
grouped_states_df = new_sector_data.groupby(['State', 'Sector'])['Employees'].sum()
grouped_states_df =grouped_states_df.reset_index()
grouped_states_df.head()

,State,Sector,Employees
0,AL,Financials,21714
1,AL,Health Care,30935
2,AL,Materials,8287
3,AR,Energy,1128
4,AR,"Food, Beverages & Tobacco",122000


In [53]:
grouped_df_2 = grouped_states_df.set_index('Sector')
grouped_df_2.head()

df_final_1 = grouped_df_2.groupby(['State'])[('Employees')].idxmax().reset_index()

In [54]:
grouped_df_3 = grouped_states_df.groupby(['State'])['Employees'].max()
df_final_2 = grouped_df_3.reset_index()

In [55]:
merged_StateMax = pd.merge(df_final_1, df_final_2, on='State')
merged_StateMax

,State,Employees_x,Employees_y
0,AL,Health Care,30935
1,AR,Retailing,2338300
2,AZ,Technology,99243
3,CA,Technology,1129452
4,CO,"Hotels, Restaurants & Leisure",89040
5,CT,Aerospace & Defense,210959
6,DE,Chemicals,7000
7,FL,"Hotels, Restaurants & Leisure",283729
8,GA,Transportation,432979
9,HI,Transportation,8607


In [56]:
merged_StateMax['State & Sector'] = merged_StateMax['State'].str.cat(merged_StateMax['Employees_x'],sep=" -- ")
merged_StateMax

,State,Employees_x,Employees_y,State & Sector
0,AL,Health Care,30935,AL -- Health Care
1,AR,Retailing,2338300,AR -- Retailing
2,AZ,Technology,99243,AZ -- Technology
3,CA,Technology,1129452,CA -- Technology
4,CO,"Hotels, Restaurants & Leisure",89040,"CO -- Hotels, Restaurants & Leisure"
5,CT,Aerospace & Defense,210959,CT -- Aerospace & Defense
6,DE,Chemicals,7000,DE -- Chemicals
7,FL,"Hotels, Restaurants & Leisure",283729,"FL -- Hotels, Restaurants & Leisure"
8,GA,Transportation,432979,GA -- Transportation
9,HI,Transportation,8607,HI -- Transportation


In [57]:
to_drop = ['Michigan', 'Puerto Rico']

df_final_3 = merged_StateMax.query('State not in @to_drop')
df_final_4 = df_final_3.reset_index()

In [58]:
df_final_4.rename(columns={'Employees_x':'Sector',
                          'Employees_y':'# Employees'}, 
                 inplace=True)

df_final_5 = df_final_4

In [59]:
df_final_5.to_csv("Resources/Final_Dataframe.csv", index=False)

In [60]:
# End of Dhruv's clean up code ------------------------------------------